In [1]:
%%configure -f
{
    "conf":  {       
             "spark.sql.catalog.glue_catalog1":"org.apache.iceberg.spark.SparkCatalog",
             "spark.sql.catalog.glue_catalog1.warehouse":"s3://jason-dremio-product-us-west-2/iceberg/glue_catalog1/tables/",
             "spark.sql.catalog.glue_catalog1.catalog-impl":"org.apache.iceberg.aws.glue.GlueCatalog",
             "spark.sql.catalog.glue_catalog1.io-impl":"org.apache.iceberg.aws.s3.S3FileIO",
             "spark.sql.catalog.glue_catalog1.lock-impl":"org.apache.iceberg.aws.glue.DynamoLockManager",
             "spark.sql.catalog.glue_catalog1.lock.table":"myGlueLockTable",
             "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
           } 
}

In [5]:
%%sql 

use glue_catalog1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [6]:
%%sql
use streamdb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [7]:
%%sh 

aws s3 ls s3://jason-dremio-product-us-west-2/iceberg-webinar/glue_catalog1/tables/streamdb.db/streaming_trips_table/data/

2022-08-15 06:41:03      11487 00003-202-8e867582-fa7e-4867-9385-b0bb53998564-00001.parquet
2022-08-15 06:41:16       8093 00003-627-058b2988-5205-4472-a382-0ea2268ba1ce-00001.parquet
2022-08-15 06:41:03       7176 00014-203-c548021f-e2ec-421e-8ac7-ef19d2977ab8-00001.parquet
2022-08-15 06:41:15       5170 00014-612-74e05696-c9b0-4859-8545-bcfacd697cf1-00001.parquet
2022-08-15 06:41:03       6361 00018-204-bd0f71ea-0e02-4269-bbab-80f95a71c2aa-00001.parquet
2022-08-15 06:41:16       4905 00018-623-89a4e9c8-099c-4515-9138-ff1a985b486d-00001.parquet
2022-08-15 06:41:03       5724 00021-205-071bdc71-6e68-4499-88ba-146d616760a7-00001.parquet
2022-08-15 06:41:16       4888 00021-636-de4dbeb8-73fc-4172-8c15-d1e5daed7e79-00001.parquet
2022-08-15 06:41:03      14020 00026-206-6749c895-9fa6-4c14-8756-f229d0a9c6cb-00001.parquet
2022-08-15 06:41:16      10175 00026-628-83a1cbbb-3dd3-4c95-8057-8773991b7b1e-00001.parquet
2022-08-15 06:41:03       4386 00027-207-036f5976-369a-440f-ba1d-48aae520960b-00

**Total number of entries**

In [8]:
%%sql

select count(*) from glue_catalog1.streamdb.streaming_trips_table

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [19]:
%%sql

select count(*) from glue_catalog1.streamdb.streaming_trips_table

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

**Select future stop IDs for a trip**

In [20]:
%%sql

select explode(futureStopIds), startTime from glue_catalog1.streamdb.streaming_trips_table
where tripId = '016050_F..N'
order by startTime;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

**Number of trips started between two intervals**

In [18]:
%%sql 

select count(*) as numOfTrips, tripId, routeId, startTime from glue_catalog1.streamdb.streaming_trips_table 
where startTime BETWEEN '2022-08-15 00:00:00' AND '2022-08-16 00:00:00'
group by tripId, routeId, startTime order by 4 desc;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

**Get future number of stops per trip and route at the moment**

In [22]:
%%sql 

select tripId, routeId, currentTs, sum(numOfFutureStops) from
(select tripId , routeId, currentTs, numOfFutureStops,
rank() over (order by currentTs) as rnk
from glue_catalog1.streamdb.streaming_trips_table) temp
where rnk = 1
group by 1,2,3
order by 4 desc;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()